# Notebook for Working With Oracle Solaris Compliance Through its REST API

This Jupyter Notebook is aimed at showing how you can use the REST interface in Oracle Solaris to work with its [Compliance framework](https://docs.oracle.com/cd/E37838_01/html/E61020/cplov-1.html#scrolltoc). The REST API is layered on top of the [Oracle Solaris Remote Administration Deamon (RAD)](https://docs.oracle.com/cd/E37838_01/html/E68270/gpzxz.html#scrolltoc), and gives access to all the RAD modules through REST. This notebook is using Python to run all the tasks. </br></br>
In this notebook we'll show how to connect to the REST interface and how to perform the various tasks you can do through REST.

> *Note 1* — You'll need to enable the `svc:/system/rad:remote` [service](https://docs.oracle.com/cd/E37838_01/html/E68270/gpzpd.html#scrolltoc) to be able to remotely connect. </br> 
> *Note 2* — If the server doesn't have a certification signed by a public CA pull in `/etc/certs/localhost/host-ca/hostca.crt` from your server and point to its location in the JSON file below. </br> 
> *Note 3* — The user you use to connect to the system will need to have either the `Compliance Assessor` or `Compliance Reporter` profiles set to be able perform these tasks. For more info see [the documentation](https://docs.oracle.com/cd/E37838_01/html/E61020/cplcmdpkg.html#OSCPLcplrights).</br>
> *Note 4* — This notebook was written with Python 3.7

Here are the steps on what to do.

## Introduction to the Compliance API

The base URI of the Compliance API is: `api/com.oracle.solaris.rad.compliance_mgr/1.0/`

The API has three main interfaces:

- **Assessment** — URI: `Assessment/` — This allows interaction with the assessments on the system, to get infomation about them, download them, and create and delete them.
- **Benchmark** — URI: `Benchmark/` — This allows interaction with the benchmarks on the system, to get information about them and their profiles.
- **Tailoring** — URI: `Tailoring/` — This allows interaction with the tailorings on the system, to get information about them, the rules they will test, and the description of the individual rules. This interface can also be used to get info further about profiles and the rules they will test.

This notebook will show how to use each of these interfaces in order.

---

## Imports and Setting Variables

First to import all the Python libraries:

In [1]:
import requests
import json
import base64
import os
import time
import pandas as pd

Turning off warnings thrown by Python:

In [2]:
import warnings
warnings.filterwarnings('ignore')

---
## Base Functions

Next we define the base functions used to make the connection with the REST interface. The first is to establish a session, the second is for the regular GET, PUT, POST, and DELETE tasks.

The function to establish a session:

In [3]:
def rad_rest_login(session_name, server_connection_info, base_authentication_uri):
    login_url = 'https://{0}:{1}/{2}'.format(*list(map(server_connection_info.get, ['server_name', 'server_port'])), base_authentication_uri)
    print("Logging in with this URL: " + login_url)

    try:
        response = session_name.post(login_url, json = server_connection_info['config_json'], verify = server_connection_info['cert_location'])
    except:
        print('no connection')
        response = 'empty'
    
    return response

The function to run regular requests:

In [4]:
def rad_rest_request(request_type, session_name, server_connection_info, rad_rest_uri, payload = None):
    
    # Create a list of variables
    query_vars = list(map(server_connection_info.get, ['server_name', 'server_port']))
    query_vars.append(rad_rest_uri)

    # Create query url to be used
    query_url = 'https://{0}:{1}/{2}'.format(*query_vars)

    response = session_name.request(request_type, query_url, json = payload)
    
    return response

---

## Load Remote System Connection Information

Now define the location of the remote system connection information:

In [5]:
config_filename = '../base_login_info.json'

The system specific information is located in a JSON file located in the same directory as the notebook, and is structured like this: </br>

```json
{
    "server_name": "<ip address>",
    "server_port": "<RAD Remote port>",
    "cert_location": "<path_to_cert>",
    "config_json": {
        "username": "<username>",
        "password": "<password>",
        "scheme": "pam",
        "preserve": true,
        "timeout": -1
    }
}
``` 

We advise to securely connect to the remote server over https however this will require the use of a certificate, in many cases the remote server will have a certificate signed by a public CA, if so please set `"cert_location": true`. If, for example in the case of a demo system, the certificate isn't signed download the `/etc/certs/localhost/host-ca/hostca.crt` file and refer to it's location with `"cert_location": "<path_to_cert>"`. If you don't want to use a certificate at all set `"cert_location": false` instead of the location of the cert file. Note this last option doesn't validate that you're connecting to correct server.

Loading this system specific information:

In [6]:
if config_filename:
    with open(config_filename, 'r') as f:
        server_connection_info = json.load(f)
        
server_connection_info['server_name']

't8ldom3.us.oracle.com'

---
---
## Assessments

The following section now shows the various commands you can run. First it'll explore the `Assessment/` part of the API, this handles all the things related to the assessments, their info, creation, deletion, etc.

Next define the base authentication URI, and the base compiance and assessment URIs:

In [7]:
base_authentication_uri = 'api/authentication/1.0/Session/'

base_compliance_uri = 'api/com.oracle.solaris.rad.compliance_mgr/1.0/'
compliance_assessment_uri = '{}Assessment/'.format(base_compliance_uri)

### Logging in and setting up a session

First establish the session and bring in the necessary credentials. Note we're printing the results to give a better insight on what's coming back, we won't be doing this for every request just a few to give you a feel:

In [8]:
s = requests.Session()

login_answer = rad_rest_login(s, server_connection_info, base_authentication_uri)

# printing the answers coming back to show the proces.
print(login_answer.status_code)
print(login_answer.text)

Logging in with this URL: https://t8ldom3.us.oracle.com:6788/api/authentication/1.0/Session/
201
{
        "status": "success",
        "payload": {
                "href": "/api/com.oracle.solaris.rad.authentication/1.0/Session/_rad_reference/3584"
        }
}


---
### Getting a list of all the current reports on the system

Now to get a list of all the current reports on the system, first we query the generic URI which gives back a list of `href` objects. These are the individual report URIs which we can use to get more information about the reports and even to fetch the reports. This is assuming there are already some reports on the system.</br></br>
First we fetch the list of reports:

In [9]:
query_answer = rad_rest_request('GET', s, server_connection_info, compliance_assessment_uri)
query_text = json.loads(query_answer.text)

print(query_answer.status_code, '\n')
print(query_text)

200 

{'status': 'success', 'payload': [{'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/6b864ff2-eddd-11ea-9126-9d4367ceef56'}, {'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/0a763c40-edba-11ea-9119-9d4367ceef56'}, {'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/fdd900fe-edc2-11ea-911d-9d4367ceef56'}, {'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/a7167964-edb7-11ea-9118-9d4367ceef56'}, {'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/12da786a-edba-11ea-911a-9d4367ceef56'}, {'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/6b3d113c-edda-11ea-9124-9d4367ceef56'}, {'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/70d56ce8-edc6-11ea-911f-9d4367ceef56'}, {'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/59535fe4-edbc-11ea-911c-9d4367ceef56'}, {'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/20b38cfe-edb7-11ea-9117-9d4367ceef56'

---
Now we have the report URI's we can use these to fetch more information about each URI. This next code runs through the list, pulls out the URI and then uses `/metadata` appended to the end to get extra information about the reports. We've added a `for` loop to iterate through each element in the response and print them out nicely:

In [10]:
for element in query_text['payload']:
    print('Getting metadata for: {}'.format(element['href'].split('/')[-1]))
    element_answer = rad_rest_request('GET', s, server_connection_info, '{}{}/metadata'.format(compliance_assessment_uri, element['href'].split('/')[-1]))
    element_text = json.loads(element_answer.text)
    
print('\nPrinting full metadata for: {}'.format(element['href'].split('/')[-1]))   
for key, item in element_text['payload'].items():
    print(key, ':', item)
print()

Getting metadata for: 6b864ff2-eddd-11ea-9126-9d4367ceef56
Getting metadata for: 0a763c40-edba-11ea-9119-9d4367ceef56
Getting metadata for: fdd900fe-edc2-11ea-911d-9d4367ceef56
Getting metadata for: a7167964-edb7-11ea-9118-9d4367ceef56
Getting metadata for: 12da786a-edba-11ea-911a-9d4367ceef56
Getting metadata for: 6b3d113c-edda-11ea-9124-9d4367ceef56
Getting metadata for: 70d56ce8-edc6-11ea-911f-9d4367ceef56
Getting metadata for: 59535fe4-edbc-11ea-911c-9d4367ceef56
Getting metadata for: 20b38cfe-edb7-11ea-9117-9d4367ceef56
Getting metadata for: 32eadbbc-edbb-11ea-911b-9d4367ceef56
Getting metadata for: 9ece4f64-eddc-11ea-9125-9d4367ceef56
Getting metadata for: 59eb0da6-edc3-11ea-911e-9d4367ceef56
Getting metadata for: a28e2f64-edb6-11ea-9112-9d4367ceef56
Getting metadata for: 1a87f678-edc9-11ea-9122-9d4367ceef56
Getting metadata for: 21d244aa-edca-11ea-9123-9d4367ceef56

Printing full metadata for: 21d244aa-edca-11ea-9123-9d4367ceef56
Name : my_tailoring.2020-09-03,02:44
Node : t8ldo

---
This does the same but now pulls the data into a Pandas dataframe:

In [11]:
query_df = pd.DataFrame()
for element in query_text['payload']:
    element_answer = rad_rest_request('GET', s, server_connection_info, '{}{}/metadata'.format(compliance_assessment_uri, element['href'].split('/')[-1]))
    query_series = pd.read_json(element_answer.text)
    query_df = query_df.append(query_series['payload'], ignore_index = True) 

The Pandas dataframe give a nice overview of the assessments:

In [12]:
query_df

,Architecture,Benchmark,Name,Node,Platform,Status,Tailoring,Timestamp,UUID,UserID,Username,Profile
0,sun4v,solaris,"my_tailoring.2020-09-03,05:03",t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,my_tailoring,2020-09-03T05:03:20,6b864ff2-eddd-11ea-9126-9d4367ceef56,0,root,NaN
1,sun4v,solaris,my_assessment,t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,NaN,2020-09-03T00:52:05,0a763c40-edba-11ea-9119-9d4367ceef56,0,root,Baseline
2,sun4v,pci-dss,"pci-dss.2020-09-03,01:53",t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,NaN,2020-09-03T02:08:31,fdd900fe-edc2-11ea-911d-9d4367ceef56,0,root,Solaris_PCI-DSS
3,sun4v,solaris,"solaris.Baseline.2020-09-03,00:32",t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,NaN,2020-09-03T00:34:55,a7167964-edb7-11ea-9118-9d4367ceef56,0,root,Baseline
4,sun4v,solaris,"solaris.Baseline.2020-09-03,00:50",t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,NaN,2020-09-03T00:51:33,12da786a-edba-11ea-911a-9d4367ceef56,0,root,Baseline
5,sun4v,solaris,"my_tailoring.2020-09-03,04:41",t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,my_tailoring,2020-09-03T04:54:13,6b3d113c-edda-11ea-9124-9d4367ceef56,0,root,NaN
6,sun4v,solaris,"my_tailoring.2020-09-03,02:18",t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,my_tailoring,2020-09-03T02:31:15,70d56ce8-edc6-11ea-911f-9d4367ceef56,0,root,NaN
7,sun4v,solaris,my_assessment,t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,NaN,2020-09-03T01:08:37,59535fe4-edbc-11ea-911c-9d4367ceef56,0,root,Baseline
8,sun4v,solaris,my_assessment,t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,NaN,2020-09-03T00:31:21,20b38cfe-edb7-11ea-9117-9d4367ceef56,0,root,Baseline
9,sun4v,pci-dss,"pci-dss.2020-09-03,00:58",t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,NaN,2020-09-03T01:13:17,32eadbbc-edbb-11ea-911b-9d4367ceef56,0,root,Solaris_PCI-DSS


Once you pull this data into Pandas it becomes very easy to find and select specific assessments by their characteristics, especially at scale.

For example if you just want to find the assessments that contain `'solaris.Baseline'` in their name you filter it this way:

In [13]:
query_df[query_df['Name'].str.contains('solaris.Baseline')]

,Architecture,Benchmark,Name,Node,Platform,Status,Tailoring,Timestamp,UUID,UserID,Username,Profile
3,sun4v,solaris,"solaris.Baseline.2020-09-03,00:32",t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,NaN,2020-09-03T00:34:55,a7167964-edb7-11ea-9118-9d4367ceef56,0,root,Baseline
4,sun4v,solaris,"solaris.Baseline.2020-09-03,00:50",t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,NaN,2020-09-03T00:51:33,12da786a-edba-11ea-911a-9d4367ceef56,0,root,Baseline
12,sun4v,solaris,"solaris.Baseline.2020-09-03,00:25",t8ldom3.us.oracle.com,cpe:/o:oracle:solaris:11,Complete,NaN,2020-09-03T00:28:34,a28e2f64-edb6-11ea-9112-9d4367ceef56,0,root,Baseline


This you could use any value in the table to search on or sort the data with.

---
Next we use one of the reports — the one that happens to be the first in the list — and use the `/contents` method to fetch the full report for this assessment. The full report contains `report.html`, `state`, `log`, and `results.xccdf.xml` as individual elements, we first have to decode it from the `base64` encoding it's in and can then save the file to disk in the `assessments` directory. For connvenience we're creating a subdirectory named after the assessment's UUID:

In [14]:
test_assessment = query_text['payload'][0]['href'].split('/')[-1]
print('getting the report for {}:'.format(test_assessment))
report_answer = rad_rest_request('GET', s, server_connection_info, '{}{}/contents'.format(compliance_assessment_uri, test_assessment))
report_text = json.loads(report_answer.text)

assessment_path = 'assessments/{}'.format(test_assessment)
os.makedirs(assessment_path, exist_ok = True)

for key in report_text['payload']:
    print('Saving {}'.format(key))
    file_name = '{}/{}'.format(assessment_path, key)
    with open(file_name, 'wb') as file:
        file.write(base64.b64decode(report_text['payload'][key]))

getting the report for 6b864ff2-eddd-11ea-9126-9d4367ceef56:
Saving report.html
Saving state
Saving log
Saving results.xccdf.xml


This an easy way of pulling across the reports you're looking for over REST.

---

### Starting a new assessment

This section shows how to kick off a new compliance assessment. 

* first this happens with a `POST` command to create the assessment 
* we pull in the UUID of the new assessment
* then we check if it was created successfully
* and then there’s a `PUT` command to set it’s state to `“AS_RUNNING”`
* once this has been set the compliance assessment will start and we can track it's progress

---

In case we want to use the current datetime in the assessment name we create a function that can generate it:

In [15]:
def get_time_stamp():
    time_zone = timezone(timedelta(hours = -7))
    return datetime.now(tz = time_zone).strftime('%Y-%m-%d,%H:%M')

Generating a dict with the benchmark, profile, and name for the assessment:

In [16]:
benchmark = 'solaris'
profile = 'Baseline'
tailoring = 'my_tailoring'

# the option to choose the assessment name
name = 'my_assessment'
# name = '{}.{}.{}'.format(benchmark, profile, get_time_stamp())

json_body = {'tailoring' : tailoring, 'name': name}
# json_body = {'benchmark': benchmark , 'profile' : profile, 'name': name}
json_body

{'tailoring': 'my_tailoring', 'name': 'my_assessment'}

---
Now we use a `POST` command to create the assessment. In return we receive the `href` reference for the newly created assessment.

In [17]:
post_response = rad_rest_request('POST', s, server_connection_info, compliance_assessment_uri, json_body)
post_response_text = json.loads(post_response.text)
post_response_uri = post_response_text['payload']['href'][1:]

print(post_response.status_code)
print(post_response_text)
print(post_response_uri)

201
{'status': 'success', 'payload': {'href': '/api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/_rad_reference/6145'}}
api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/_rad_reference/6145


---
We can now fetch the assessment UUID by using this `href` reference. We want to do this because the reference is temporary where the UUID is fixed.

In [18]:
request_uuid = json.loads(rad_rest_request('GET', s, server_connection_info, '{}/uuid'.format(post_response_uri)).text)
print(request_uuid)

post_response_uuid = '{}{}'.format(compliance_assessment_uri, request_uuid['payload'])
print(post_response_uuid)

{'status': 'success', 'payload': '50609b94-eed5-11ea-913a-9d4367ceef56'}
api/com.oracle.solaris.rad.compliance_mgr/1.0/Assessment/50609b94-eed5-11ea-913a-9d4367ceef56


---
Once we have the tthe assessment UUID we can check it's current state, this should be `"As_CREATING"`:

In [19]:
get_state = rad_rest_request('GET', s, server_connection_info, '{}/state'.format(post_response_uuid))

print(json.loads(get_state.text))

{'status': 'success', 'payload': 'AS_CREATING'}


---
To now start the assessment we set it's state to `"AS_RUNNING"`. Note this is to the same `/state` URI, but this time instead of using `GET` we're using `PUT`:

In [20]:
json_body_running = {"value": 'AS_RUNNING'}

put_response = rad_rest_request('PUT', s, server_connection_info, '{}/state'.format(post_response_uuid), json_body_running)
print(put_response.status_code)
print(json.loads(put_response.text))

200
{'status': 'success', 'payload': 'AS_RUNNING'}


---
The assessment should now be running we can check this with this loop. Currently the status isn't refreshed for the running session so we start a new session and run the loop:

In [21]:
while True:
    s = requests.Session()
    login_again = rad_rest_login(s, server_connection_info, base_authentication_uri)
    
    my_list = rad_rest_request('GET', s, server_connection_info, '{}/metadata'.format(post_response_uuid))
    my_text = json.loads(my_list.text)
    print(my_text['payload']['Status'])
    if my_text['payload']['Status'] != 'Running':
        print('done')
        break
    time.sleep(15)

Logging in with this URL: https://t8ldom3.us.oracle.com:6788/api/authentication/1.0/Session/
Running
Logging in with this URL: https://t8ldom3.us.oracle.com:6788/api/authentication/1.0/Session/
Running
Logging in with this URL: https://t8ldom3.us.oracle.com:6788/api/authentication/1.0/Session/
Complete
done


---
### Deleting an assessment

This section shows how to delete an assessment. 

First we take the UUID of the report we just created:

In [22]:
assessment_uuid = request_uuid['payload']

Then we run the `DELETE` commannd:

In [23]:
delete_response = rad_rest_request('DELETE', s, server_connection_info, '{}{}'.format(compliance_assessment_uri, assessment_uuid))

print(delete_response.status_code)
print(json.loads(delete_response.text))

200
{'status': 'success', 'payload': None}


---
---

## Benchmarks

The next sections will go into the `Benchmark/` part of the REST API.

First we set the new URI for this part of the API:

In [24]:
compliance_benchmark_uri = '{}Benchmark/'.format(base_compliance_uri)
compliance_benchmark_uri

'api/com.oracle.solaris.rad.compliance_mgr/1.0/Benchmark/'

---
The following command will allow you to get the available benchmarks on the system.

In [25]:
benchmark_get = rad_rest_request('GET', s, server_connection_info, '{}'.format(compliance_benchmark_uri))
benchmark_list = json.loads(benchmark_get.text)

print(benchmark_get.status_code)
print(benchmark_list)

200
{'status': 'success', 'payload': [{'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Benchmark/pci-dss'}, {'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Benchmark/solaris'}]}


---
Next we explore the information linked to these benchmarks: `/name`, `/title`, `/profiles`

In [26]:
for benchmark_uri in benchmark_list['payload']:
    print(benchmark_uri['href'])
    benchmark_name = json.loads(rad_rest_request('GET', s, server_connection_info, '{}/name'.format(benchmark_uri['href'])).text)
    print(benchmark_name)
    benchmark_title = json.loads(rad_rest_request('GET', s, server_connection_info, '{}/title'.format(benchmark_uri['href'])).text)
    print(benchmark_title)
    benchmark_profiles = json.loads(rad_rest_request('GET', s, server_connection_info, '{}/profiles'.format(benchmark_uri['href'])).text)
    print(benchmark_profiles)

api/com.oracle.solaris.rad.compliance_mgr/1.0/Benchmark/pci-dss
{'status': 'success', 'payload': 'pci-dss'}
{'status': 'success', 'payload': 'PCI-DSS Security/Compliance benchmark for Oracle Solaris'}
{'status': 'success', 'payload': ['Solaris_PCI-DSS']}
api/com.oracle.solaris.rad.compliance_mgr/1.0/Benchmark/solaris
{'status': 'success', 'payload': 'solaris'}
{'status': 'success', 'payload': 'Oracle Solaris Security Policy'}
{'status': 'success', 'payload': ['Baseline', 'Recommended']}


---
---

## Tailoring

The `Tailoring/` API is closely related to the `Benchmark/` API. With this API you can find out if there are tailorings on the system and if so what they  are called and how they are tailored. I.e. which rules are set for this tailoring. Incidentally you can also use this API to find the way the predefined profiles are set.

First we set the tailoring URI:

In [27]:
compliance_tailoring_uri = '{}Tailoring/'.format(base_compliance_uri)
compliance_tailoring_uri

'api/com.oracle.solaris.rad.compliance_mgr/1.0/Tailoring/'

---
Then we get the list of available tailorings:

In [28]:
tailoring_get = rad_rest_request('GET', s, server_connection_info, '{}'.format(compliance_tailoring_uri))

print(tailoring_get.status_code)
tailoring_list = json.loads(tailoring_get.text)
print(tailoring_list)

200
{'status': 'success', 'payload': [{'href': 'api/com.oracle.solaris.rad.compliance_mgr/1.0/Tailoring/my_tailoring'}]}


---
We can now use this to get the full list of rules asigned to this tailoring:

In [29]:
tailoring_dfs = {}

for tailoring in tailoring_list['payload']:
    print(tailoring['href'])
    json_tailoring = {'tailoring': tailoring['href']}
    tailoring_description = json.loads(rad_rest_request('PUT', s, server_connection_info, 
                                                        '{}{}/_rad_method/get_descriptions'.format(compliance_benchmark_uri, 'solaris'), 
                                                        json_tailoring).text)
#     print(tailoring_description)
    tailoring_dfs[tailoring['href'].split('/')[-1]] = pd.DataFrame(tailoring_description['payload'])

api/com.oracle.solaris.rad.compliance_mgr/1.0/Tailoring/my_tailoring


---

Because there can be multiple tailorings on the system, I've put the dataframes inside a dict:

In [30]:
tailoring_dfs.keys()

dict_keys(['my_tailoring'])

Now we can look at the first five rules for each tailoring:

In [31]:
for item in tailoring_dfs.keys():
    print(tailoring_dfs[item].head())

      ruleid                                             title  \
0  OSC-54005                     Package integrity is verified   
1  OSC-53005                         The OS version is current   
2  OSC-53015                  Required CVE fixes are installed   
3  OSC-53505  Package signature checking is globally activated   
4  OSC-16005                     All local filesystems are ZFS   

                                         description  
0  Run 'pkg verify' to check that\n        all in...  
1  Systems should be kept up to date to ensure\n ...  
2  For the set of packages present on the system ...  
3  Package signature checking should be globally ...  
4  ZFS is the default filesystem for Oracle Solar...  


---
Finally we can also use this API to get the list of rules set for the built-in profiles and pull into a Pandas dataframe:

In [32]:
profile_description = json.loads(rad_rest_request('PUT', s, server_connection_info, 
                                                  '{}{}/_rad_method/get_descriptions'.format(compliance_benchmark_uri, 'solaris'), 
                                                  {'tailoring': 'Baseline'}).text)

profile_df = pd.DataFrame(profile_description['payload'])

Showing the newly created dataframe:

In [33]:
profile_df

,ruleid,title,description
0,OSC-54005,Package integrity is verified,Run 'pkg verify' to check that\n all in...
1,OSC-53005,The OS version is current,Systems should be kept up to date to ensure\n ...
2,OSC-53015,Required CVE fixes are installed,For the set of packages present on the system ...
3,OSC-53505,Package signature checking is globally activated,Package signature checking should be globally ...
4,OSC-16005,All local filesystems are ZFS,ZFS is the default filesystem for Oracle Solar...
...,...,...,...
246,OSC-06110,Cluster authentication protocol is des and key...,Using the claccess command to check if authent...
247,OSC-06111,Cluster resource_security set to SECURE,Cluster resource_security property should be s...
248,OSC-06112,Cluster eventlog file must not be world-readable,Check the permissions of eventlog file. It mus...
249,OSC-06113,Cluster_check log files must not be world-read...,Check ownership of all the cluster_check log f...


And you can use `str.contains` to search for a specific thing inside the rule descriptions:

In [34]:
for cell in profile_df[profile_df['description'].str.contains('oscv', case = False)]['description']:
    print(cell)

The rule validates that only approved ports are allowed to be
        bound on non-loopback addresses. Any other ports that are being used,
        but have not been excluded will be reported as a failure.
        
        
        By default, there are several network services that can send and receive
        network packets on a newly-installed Oracle Solaris system, including 
        sshd(8), rpcbind(8), nfsd(8), webui-service(7), and rad(8).
        
        
        These services can be excluded based on certain values set in other rules
        which includes the following:
        
        OSCV-72011 ssh service, which is currently set to enabled.
        
        OSCV-39510 nfs-server service, which is currently set to disabled.
        
        OSCV-98511 webui-server service, which is currently set to enabled.
        
        OSCV-99011 rad-remote service, which is currently set to enabled.
        
        OSCV-34010 lpd service, which is currently set to enabled.
      

The end of this notebook